**If you haven't already done so, please take a look at this [FAQ](https://q37.info/s/kqsb3gmg), especially if you run this notebook on *Google* *Colab*.**

If the [*atlastk*](http://atlastk.org) module is not installed, it will be installed from [*Pypi*](https://q37.info/s/9srmskcm).

Don't worry, it's a very lightweight package (~20 Ko), and it has no dependencies…

In [1]:
try:
  import atlastk
except:
  !pip install atlastk
  import atlastk

atlastk.setJupyterHeight("500px") # Adjusting the height of the iframe in which the application will be displayed…

In [2]:
import IPython, base64

def show(html,height):
    return IPython.display.IFrame(f"data:text/html;base64,{base64.b64encode(html.encode()).decode('utf-8') }".replace("+","%2B"), "100%", f"{height}px")

In [3]:
BODY_CONTACT = """
<fieldset id="Contact">
  <table style="margin: auto;">
    <tr>
      <td>
        <label for="Name">Full name:</label>
      </td>
      <td>
        <input id="Name" size="50" />
      </td>
    </tr>
    <tr>
      <td>
        <label for="Address">Address:</label>
      </td>
      <td>
        <input id="Address" size="50" />
      </td>
    </tr>
    <tr>
      <td>
        <label for="Phone">Phone:</label>
      </td>
      <td>
        <input id="Phone" type="tel" size="50" />
      </td>
    </tr>
    <tr>
      <td>
        <label for="Note">Note:</label>
      </td>
      <td>
        <textarea id="Note" style="width: 100%;"></textarea>
      </td>
    </tr>
  </table>
</fieldset>
"""

In [4]:
show(BODY_CONTACT,200)

In [5]:
BODY_BUTTONS = """
<div style="display: table; margin: 10px auto auto auto;">
  <div>
    <button class="Display" data-xdh-onevent="New">New</button>
    <span class="DisplayAndSelect">
      <button data-xdh-onevent="Edit">Edit</button>
      <button data-xdh-onevent="Delete">Delete</button>
    </span>
  </div>
  <div class="Edition">
    <button data-xdh-onevent="Cancel">Cancel</button>
    <button data-xdh-onevent="Submit">Submit</button>
  </div>
</div>
"""

In [6]:
show(BODY_BUTTONS,100)

In [7]:
BODY_FORM = f"""
<fieldset>
  {BODY_CONTACT}
  {BODY_BUTTONS}
</fieldset>    
"""

In [8]:
show(BODY_FORM,300)

In [9]:
BODY_CONTACTS = """
<div style="display: table; margin: 10px auto auto auto; border-collapse: collapse;">
  <table id="Contacts" style="cursor: default; border-collapse: collapse;">
    <thead>
      <th>Name</th>
      <th>Address</th>
      <th>Phone</th>
      <th>Note</th>
    </thead>
    <tbody id="Content" />
  </table>
</div>
"""

In [10]:
show(BODY_CONTACTS,50)

In [11]:
BODY = BODY_FORM + BODY_CONTACTS

In [12]:
show(BODY,350)

In [13]:
HEAD = """
<title>Address book</title>
<link rel="icon" type="image/png"
  href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACAAAAAgBAMAAACBVGfHAAAAMFBMVEUEAvyEhsxERuS8urQsKuycnsRkYtzc2qwUFvRUVtysrrx0ctTs6qTMyrSUksQ0NuyciPBdAAABHklEQVR42mNgwAa8zlxjDd2A4POfOXPmzZkFCAH2M8fNzyALzDlzg2ENssCbMwkMOsgCa858YOjBKxBzRoHhD7LAHiBH5swCT9HQ6A9ggZ4zp7YCrV0DdM6pBpAAG5Blc2aBDZA68wCsZPuZU0BDH07xvHOmAGKKvgMP2NA/Zw7ADIYJXGDgLQeBBSCBFu0aoAPYQUadMQAJAE29zwAVWMCWpgB08ZnDQGsbGhpsgCqBQHNfzRkDEIPlzFmo0T5nzoMovjPHoAK8Zw5BnA5yDosDSAVYQOYMKIDZzkoDzagAsjhqzjRAfXTmzAQgi/vMQZA6pjtAvhEk0E+ATWRRm6YBZuScCUCNN5szH1D4TGdOoSrggtiNAH3vBBjwAQCglIrSZkf1MQAAAABJRU5ErkJggg==" />
<style>
  #Contact label {
    font-weight: bold;
  }

  #Contact span {
    text-align: left;
  }

  #Contacts th,
  #Contacts td {
    border: 1px solid black;
    padding: 0% 5px;
  }

  #Contacts td:nth-child(3) {
    text-align: right;
  }

  #Contacts tr:nth-child(even)
  {
    background: #CCC
  }

  #Contacts tr:nth-child(odd)
  {
    background: #FFF
  }

  #Contact *:disabled {
    background-color: snow;
    color: initial;
  }
</style>
<style id="HideDisplay">
  .Display {
    display: none;
  }
</style>
<style id="HideDisplayAndSelect">
  .DisplayAndSelect {
    display: none;
  }
</style>
<style id="HideEdition">
  .Edition {
    display: none;
  }
</style>
"""

In [14]:
def ac_connect(dom):
  dom.inner("",BODY)

callbacks = {
    "": ac_connect 
} 

In [15]:
atlastk.launch(callbacks,headContent = HEAD)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=feaaf147-70c6-4466-8b98-c65b6a41237e
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [16]:
EXAMPLE = [
  {
    "Name": "Holmes, Sherlock",
    "Address": "221B Baker Street, Londres",
    "Phone": "(use telegraph)",
    "Note": "Great detective!"
  },
  {
    "Name": "Holmes, Mycroft",
    "Address": "Diogenes Club, Pall Mall, Londres",
    "Phone": "(use telegraph)",
    "Note": "Works for the British government.\nBrother of Holmes, Sherlock."
  },
  {
    "Name": "Tintin",
    "Address": "Château de Moulinsart",
    "Phone": "421",
    "Note": "Has a dog named Snowy."
  },
  {
    "Name": "Tournesol, Tryphon (prof.)",
    "Address": "Château de Moulinsart",
    "Phone": "421",
    "Note": "Creator of the Bianca rose."
  }
]

contacts = EXAMPLE

In [17]:
def display_contacts(dom):
  html = ""

  for contactId in range(len(contacts)):
    contact = contacts[contactId]
    html += f'<tr id="{contactId}" data-xdh-onevent="Select" style="cursor: pointer;">'
    for key in contact:
      html += f'<td>{contact[key]}</td>'
    html += '</td>'

  dom.inner("Content", html)

In [18]:
def ac_connect(dom):
  dom.inner("",BODY)
  display_contacts(dom)
    
callbacks[""] = ac_connect

In [19]:
atlastk.launch(callbacks)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=a0ce17fa-6613-421e-bac1-b7ef68e61e3a
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [20]:
def display_contact(contactId,dom):
  dom.set_values(contacts[contactId])

In [21]:
def ac_select(dom,id):
  display_contact(int(id),dom)

callbacks["Select"] = ac_select

In [22]:
atlastk.launch(callbacks)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=ba21f337-5730-47df-83cf-b29ac1039a5b
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [23]:
FIELDS = [
  "Name",
  "Address",
  "Phone",
  "Note"
]

In [24]:
def update_outfit(dom):
  dom.disable_elements(FIELDS)
  dom.disable_element("HideDisplay")

In [25]:
def ac_connect(dom):
  dom.inner("",BODY)
  display_contacts(dom)
  update_outfit(dom)

callbacks[""] = ac_connect

In [26]:
def ac_select(dom,id):
  display_contact(int(id),dom)
  update_outfit(dom)
    
callbacks["Select"] = ac_select

In [27]:
atlastk.launch(callbacks)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=3f8c3813-4722-4ea8-bb42-4c3c7cb25914
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [28]:
import enum

class State(enum.Enum):
  DISPLAY = enum.auto() # Affichage
  EDIT = enum.auto()    # Édition

In [29]:
class Board:
  def __init__(self):
    self.state = State.DISPLAY

In [30]:
def update_outfit(board,dom):
  if board.state == State.DISPLAY:
    dom.disable_elements(FIELDS)
    dom.disable_element("HideDisplay")
  elif board.state == State.EDIT:
    dom.enable_elements(FIELDS)
    dom.enable_elements("HideDisplay")

In [31]:
def ac_connect(board,dom):
  dom.inner("",BODY)
  display_contacts(dom)
  board.state = State.DISPLAY
  update_outfit(board,dom)

callbacks[""] = ac_connect

In [32]:
def ac_select(board,dom,id):
  display_contact(int(id),dom)
  board.state = State.DISPLAY
  update_outfit(board,dom)
    
callbacks["Select"] = ac_select

In [33]:
EMPTY_CONTACT = {
  "Name": "",
  "Address": "",
  "Phone": "",
  "Note": ""
}

In [34]:
def display_contact(contactId,dom):
  dom.set_values(EMPTY_CONTACT if contactId == None else contacts[contactId])

In [35]:
def ac_new(board,dom):
  board.state = State.EDIT
  display_contact(None,dom)
  update_outfit(board,dom)
  dom.focus("Name")

callbacks["New"] = ac_new

In [36]:
atlastk.launch(callbacks,Board)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=969864af-0a72-4707-b82f-30a3854fe88c
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [37]:
def update_outfit(board,dom):
  if board.state == State.DISPLAY:
    dom.disable_elements(FIELDS)
    dom.disable_element("HideDisplay")
    dom.enable_element("HideEdition")
  elif board.state == State.EDIT:
    dom.enable_elements(FIELDS)
    dom.enable_element("HideDisplay")
    dom.disable_element("HideEdition")

In [38]:
def ac_cancel(board,dom):
  if dom.confirm("Are you sure?"):
    display_contact(None,dom)
    board.state = State.DISPLAY
    update_outfit(board,dom)
    
callbacks["Cancel"] = ac_cancel

In [39]:
def ac_submit(board,dom):
  idsAndValues = dom.get_values(FIELDS)

  if not idsAndValues['Name'].strip():
    dom.alert("The name field can not be empty!")
  else:
    board.state = State.DISPLAY
    contacts.append(idsAndValues)
    display_contact(None,dom)
    display_contacts(dom)
    update_outfit(board,dom)
    
callbacks["Submit"] = ac_submit

In [40]:
atlastk.launch(callbacks,Board)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=204387ff-4cb6-40f5-b419-c73ce0f0d59c
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [41]:
class Board:
  def __init__(self):
    self.state = State.DISPLAY
    self.contactId = None

In [42]:
def ac_select(board,dom,id):
  board.contactId = int(id)
  display_contact(board.contactId,dom)  
  board.state = State.DISPLAY
  update_outfit(board,dom)
    
callbacks["Select"] = ac_select

In [43]:
def update_outfit(board,dom):
  if board.state == State.DISPLAY:
    dom.disable_elements(FIELDS)
    dom.disable_element("HideDisplay")
    dom.enable_element("HideEdition")
    if board.contactId == None:
      dom.enable_element("HideDisplayAndSelect")
    else:
      dom.disable_element("HideDisplayAndSelect")
  elif board.state == State.EDIT:
    dom.enable_elements(FIELDS)
    dom.enable_elements(("HideDisplay","HideDisplayAndSelect"))
    dom.disable_element("HideEdition")

In [44]:
def ac_edit(board,dom):
  board.state = State.EDIT
  display_contact(board.contactId,dom)
  update_outfit(board,dom)
  dom.focus("Name")
    
callbacks["Edit"] = ac_edit

In [45]:
def ac_cancel(board,dom):
  if dom.confirm("Are you sure?"):
    display_contact(board.contactId,dom)
    board.state = State.DISPLAY
    update_outfit(board,dom)
    
callbacks["Cancel"]=ac_cancel

In [46]:
def ac_submit(board,dom):
  idsAndValues = dom.get_values(FIELDS)

  if not idsAndValues['Name'].strip():
    dom.alert("The name field can not be empty!")
  else:
    board.state = State.DISPLAY
    if board.contactId == None:
      contacts.append(idsAndValues)
    else:
      contacts[board.contactId] = idsAndValues
    display_contact(board.contactId,dom)
    display_contacts(dom)
    update_outfit(board,dom)
    
callbacks["Submit"]=ac_submit    

In [47]:
def ac_delete(board,dom):
  contacts.pop(board.contactId)
  board.contactId = None;
  display_contact(None,dom)
  display_contacts(dom)
  update_outfit(board,dom)

callbacks["Delete"] = ac_delete

In [48]:
atlastk.launch(callbacks,Board)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=09edfdc5-9bf4-4e50-ac68-1184a5f258a3
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!



In [49]:
def ac_refresh(board,dom):
  display_contacts(dom)

callbacks["Refresh"] = ac_refresh

In [50]:
def ac_submit(board,dom):
  idsAndValues = dom.get_values(FIELDS)

  if not idsAndValues['Name'].strip():
    dom.alert("The name field can not be empty!")
  else:
    board.state = State.DISPLAY
    if board.contactId == None:
      contacts.append(idsAndValues)
    else:
      contacts[board.contactId] = idsAndValues
    display_contact(board.contactId,dom)
    atlastk.broadcast_action("Refresh")
    update_outfit(board,dom)
    
callbacks["Submit"] = ac_submit

In [51]:
def ac_delete(board,dom):
  contacts.pop(board.contactId)
  board.contactId = None;
  display_contact(None,dom)
  atlastk.broadcast_action("Refresh")
  update_outfit(board,dom)

callbacks["Delete"] = ac_delete

In [52]:
atlastk.launch(callbacks,Board)

Connection to 'faas1.q37.info:53700'...
Connected to 'faas1.q37.info:53700'.

****************************************************************************
     Your application is connected to the free public server. Welcome!
     THIS SERVICE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
     EXPRESS OR IMPLIED. THE USE OF THIS SERVICE IS AT YOUR OWN RISK.
****************************************************************************
                    
http://faas1.q37.info/xdh.php?_token=13654f2c-e4d2-4c80-b819-bcfda6c806a1
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Open above URL in a web browser (click, right click or copy/paste). Enjoy!

